In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df=df.drop(['Name','PassengerId','Ticket','Cabin'],axis=1)
df.head()
df['Embarked']=df['Embarked'].fillna(method="bfill")
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      0
dtype: int64

In [4]:
X=df.iloc[:,1:].values
X

array([[3, 'male', 22.0, ..., 0, 7.25, 'S'],
       [1, 'female', 38.0, ..., 0, 71.2833, 'C'],
       [3, 'female', 26.0, ..., 0, 7.925, 'S'],
       ...,
       [3, 'female', nan, ..., 2, 23.45, 'S'],
       [1, 'male', 26.0, ..., 0, 30.0, 'C'],
       [3, 'male', 32.0, ..., 0, 7.75, 'Q']], dtype=object)

In [5]:
from sklearn.impute import SimpleImputer
impute=SimpleImputer(missing_values=np.nan,strategy="mean")

# Give me only those rows and columns which has only numeric values
# there we provided all rows, and columns from 1 to 3(excluded) since Age and Salary column contain missing values.
impute.fit(X[:,2:4])
X[:,2:4]=impute.transform(X[:,2:4])

In [6]:
X

array([[3, 'male', 22.0, ..., 0, 7.25, 'S'],
       [1, 'female', 38.0, ..., 0, 71.2833, 'C'],
       [3, 'female', 26.0, ..., 0, 7.925, 'S'],
       ...,
       [3, 'female', 29.69911764705882, ..., 2, 23.45, 'S'],
       [1, 'male', 26.0, ..., 0, 30.0, 'C'],
       [3, 'male', 32.0, ..., 0, 7.75, 'Q']], dtype=object)

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[("encoder", OneHotEncoder(),[1])],remainder="passthrough")

# Since ct.fit_transform doesn't return an array and since our model only requires array and X is also in a form of array hence 
# np.array(ct...)
X=np.array(ct.fit_transform(X))

In [8]:
X

array([[0.0, 1.0, 3, ..., 0, 7.25, 'S'],
       [1.0, 0.0, 1, ..., 0, 71.2833, 'C'],
       [1.0, 0.0, 3, ..., 0, 7.925, 'S'],
       ...,
       [1.0, 0.0, 3, ..., 2, 23.45, 'S'],
       [0.0, 1.0, 1, ..., 0, 30.0, 'C'],
       [0.0, 1.0, 3, ..., 0, 7.75, 'Q']], dtype=object)

In [9]:
ct=ColumnTransformer(transformers=[("encoder", OneHotEncoder(),[-1])],remainder="passthrough")

# Since ct.fit_transform doesn't return an array and since our model only requires array and X is also in a form of array hence 
# np.array(ct...)
X=np.array(ct.fit_transform(X))

In [10]:
X

array([[0.0, 0.0, 1.0, ..., 1.0, 0, 7.25],
       [1.0, 0.0, 0.0, ..., 1.0, 0, 71.2833],
       [0.0, 0.0, 1.0, ..., 0.0, 0, 7.925],
       ...,
       [0.0, 0.0, 1.0, ..., 1.0, 2, 23.45],
       [1.0, 0.0, 0.0, ..., 0.0, 0, 30.0],
       [0.0, 1.0, 0.0, ..., 0.0, 0, 7.75]], dtype=object)

In [11]:
Y=df.iloc[:,0]
Y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=1)

In [13]:
from sklearn import tree
model = tree.DecisionTreeClassifier()
model.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [14]:
model.score(X_train,y_train)

0.9873595505617978

In [15]:
model.predict(X_test)

array([1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 1], dtype=int64)